In [ ]:
import geopandas as gpd
import os
import numpy as np
import rasterio
from rasterio.mask import mask
import time

In [ ]:
#read the polygons
k45_mines = gpd.read_file("data/polygons/global_mining_polygons_v2.gpkg")

k45_mines = k45_mines.set_geometry("geometry") #set the column called 'geometry' as the geometry of the GeoPandas
k45_mines = k45_mines.to_crs("EPSG:4326") #wgs84

k45_mines.head()

In [ ]:
all_tif_mosaics = sorted(os.listdir('/Users/flurin/Downloads/mosaic/'))
all_tif_mosaics

#If list includes .DS_Store file move to directory in terminal and use <find . -name '.DS_Store' -type f -delete> to remove it

In [ ]:
for i in range(len(all_tif_mosaics)):
    file = all_tif_mosaics[i]
    #print('data/viirs-tif/mosaic/' + file)
    viirs_data = rasterio.open('/Users/flurin/Downloads/mosaic/' + file)

    buff_2km_mean = []
    buff_2km_std = []
    
    for j in range(len(k45_mines)):
        active_polygon = k45_mines.iloc[j]
        buffer_shape = k45_mines.iloc[j]['geometry'].buffer(0.0166).difference(k45_mines.iloc[j]['geometry'])
        crop_img, crop_transform = mask(viirs_data, shapes = [buffer_shape], crop=True)

        buff_2km_mean.append(np.mean(crop_img))
        buff_2km_std.append(np.std(crop_img))
        
        #print(f'for mine nr. {j} in year {i} the 2km buffer mean is {np.mean(crop_img)} and the std is {np.std(crop_img)}')

    k45_mines['mean_2km_buff_' + all_tif_mosaics[i][:4]] = buff_2km_mean
    k45_mines['std_2km_buff_' + all_tif_mosaics[i][:4]] = buff_2km_std
    
    print(f'Year {all_tif_mosaics[i][:4]} is evaluated.')

In [ ]:
for i in range(len(all_tif_mosaics)):
    file = all_tif_mosaics[i]
    #print('data/viirs-tif/mosaic/' + file)
    viirs_data = rasterio.open('/Users/flurin/Downloads/mosaic/' + file)

    buff_10km_mean = []
    buff_10km_std = []
    
    for j in range(len(k45_mines)):
        active_polygon = k45_mines.iloc[j]
        buffer_shape = k45_mines.iloc[j]['geometry'].buffer(0.083).difference(k45_mines.iloc[j]['geometry'].buffer(0.0166))
        crop_img, crop_transform = mask(viirs_data, shapes = [buffer_shape], crop=True)

        buff_10km_mean.append(np.mean(crop_img))
        buff_10km_std.append(np.std(crop_img))
        
        #print(f'for mine nr. {j} in year {i} the 2km buffer mean is {np.mean(crop_img)} and the std is {np.std(crop_img)}')

    k45_mines['mean_10km_buff_' + all_tif_mosaics[i][:4]] = buff_10km_mean
    k45_mines['std_10km_buff_' + all_tif_mosaics[i][:4]] = buff_10km_std
    
    print(f'Year {all_tif_mosaics[i][:4]} is evaluated.')

In [ ]:
k45_mines.head()

In [ ]:
k45_mines.columns

In [ ]:
header = ['AREA' , 'mean_2km_buff_2012',
       'std_2km_buff_2012', 'mean_2km_buff_2013', 'std_2km_buff_2013',
       'mean_2km_buff_2014', 'std_2km_buff_2014', 'mean_2km_buff_2015',
       'std_2km_buff_2015', 'mean_2km_buff_2016', 'std_2km_buff_2016',
       'mean_2km_buff_2017', 'std_2km_buff_2017', 'mean_2km_buff_2018',
       'std_2km_buff_2018', 'mean_2km_buff_2019', 'std_2km_buff_2019',
       'mean_2km_buff_2020', 'std_2km_buff_2020', 'mean_2km_buff_2021',
       'std_2km_buff_2021', 'mean_10km_buff_2012', 'std_10km_buff_2012',
       'mean_10km_buff_2013', 'std_10km_buff_2013', 'mean_10km_buff_2014',
       'std_10km_buff_2014', 'mean_10km_buff_2015', 'std_10km_buff_2015',
       'mean_10km_buff_2016', 'std_10km_buff_2016', 'mean_10km_buff_2017',
       'std_10km_buff_2017', 'mean_10km_buff_2018', 'std_10km_buff_2018',
       'mean_10km_buff_2019', 'std_10km_buff_2019', 'mean_10km_buff_2020',
       'std_10km_buff_2020', 'mean_10km_buff_2021', 'std_10km_buff_2021']
k45_mines.to_csv('bufferdata_actual.csv', columns = header)